In [ ]:
from pdfminer.high_level import extract_text
import re
import pandas as pd
from nameparser import HumanName
from tqdm import tqdm

pdf_path = "../data/ERC-2024-AdG-panel-members.pdf"

text = extract_text(pdf_path)

lines = [l.strip() for l in text.split("\n") if l.strip()]

results = []
current_domain = None
current_subdomain = None
subdomain_queue = []

for line in tqdm(lines, desc='Processing', total=len(lines)):
    # Domain
    if "PHYSICAL SCIENCES AND ENGINEERING" in line:
        current_domain = "PE"
        continue
    elif "LIFE SCIENCES" in line:
        current_domain = "LS"
        continue
    elif "SOCIAL SCIENCES AND HUMANITIES" in line:
        current_domain = "SH"
        continue

    # Subdomain(s) erkennen (inkl. geschlossener Klammer)
    sub_matches = re.findall(r"([A-Za-z ,:'\-&]+?\((?:PE|LS|SH)\d+\))", line)
    if sub_matches:
        for sm in sub_matches:
            subdomain_queue.append(sm.strip())
        continue

    # Personen
    if "(Panel Chair)" in line:
        if subdomain_queue:
            current_subdomain = subdomain_queue.pop(0)
        name = line.replace("(Panel Chair)", "").strip()
        ptype = "Chair"
    else:
        name = line.strip()
        ptype = "Member"

    if current_domain and current_subdomain and name:
        hn = HumanName(name)
        results.append({
            "Domain": current_domain,
            "Subdomain": current_subdomain,
            "Forename": hn.first,
            "Lastname": hn.last,
            "Fullname": name,
            "Type": ptype
        })

df = pd.DataFrame(results)

# entferne Reihen, deren Fullname "ERC-" oder Seitenangaben wie "p.8/8" (oder generell "p.<num>/<num>") enthält
bad_mask = df['Fullname'].str.contains(r'ERC-|p\.\s*\d+/\d+', case=False, na=False, regex=True)
# optional: anzeigen, was entfernt würde
display(df[bad_mask])
# tatsächliches Entfernen
df = df[~bad_mask].reset_index(drop=True)

# Testausgabe
df_filter = df.loc[:, ['Forename', 'Lastname', 'Fullname', 'Type']]

# if ERC in Forename then delete row



display(df_filter)

#




Processing: 100%|██████████| 487/487 [00:00<00:00, 11351.08it/s]



,Domain,Subdomain,Forename,Lastname,Fullname,Type
55,PE,Physical and Analytical Chemical Sciences (PE4),ERC-2024-AdG,p.2/8,ERC-2024-AdG Panel Members List – Release date...,Member
122,PE,Products and Processes Engineering (PE8),ERC-2024-AdG,p.3/8,ERC-2024-AdG Panel Members List – Release date...,Member
169,PE,Materials Engineering (PE11),ERC-2024-AdG,p.4/8,ERC-2024-AdG Panel Members List – Release date...,Member
225,LS,"Physiology in Health, Disease and Ageing (LS4)",ERC-2024-AdG,p.5/8,ERC-2024-AdG Panel Members List – Release date...,Member
287,LS,"Environmental Biology, Ecology and Evolution (...",ERC-2024-AdG,p.6/8,ERC-2024-AdG Panel Members List – Release date...,Member
301,LS,Biotechnology and Biosystems Engineering (LS9),ERC-2024-AdG,p.7/8,ERC-2024-AdG Panel Members List – Release date...,Member
366,SH,The Human Mind and Its Complexity (SH4),ERC-2024-AdG,p.7/8,ERC-2024-AdG Panel Members List – Release date...,Member
434,SH,Studies of Cultures and Arts (SH8),ERC-2024-AdG,p.8/8,ERC-2024-AdG Panel Members List – Release date...,Member


,Forename,Lastname,Fullname,Type
0,Sara,van de Geer,Sara van de Geer,Chair
1,Elisabeth,Bouscaren,Elisabeth Bouscaren,Member
2,Lucia,Caporaso,Lucia Caporaso,Member
3,Alessandra,Celletti,Alessandra Celletti,Member
4,Anthony,Davison,Anthony Davison,Member
...,...,...,...,...
422,Sarah,Pink,Sarah Pink,Member
423,Emilio,Ros-Fábregas,Emilio Ros-Fábregas,Member
424,Katrin,Tiidenberg,Katrin Tiidenberg,Member
425,Sahana,Udupa,Sahana Udupa,Member


In [ ]:
domains = [
    "PHYSICAL SCIENCES AND ENGINEERING (PE)",
    "LIFE SCIENCES (LS)",
    "SOCIAL SCIENCES AND HUMANITIES (SH)",
]

subdomains = [
    "Mathematics (PE1)",
    "Fundamental Constituents of Matter (PE2)",
    "Condensed Matter Physics (PE3)",
    "Physical and Analytical Chemical Sciences (PE4)",
    "Synthetic Chemistry and Materials (PE5)",
    "Computer Science and Informatics (PE6)",
    "Systems and Communication Engineering (PE7)",
    "Products and Processes Engineering (PE8)",
    "Universe Sciences (PE9)",
    "Earth System Science (PE10)",
    "Materials Engineering (PE11)", 
    "Molecules of Life: Biological Mechanisms, Structures and Functions (LS1)",
    "Integrative Biology: From Genes and Genomes to Systems (LS2)",
    "Cell Biology, Development, Stem Cells and Regeneration (LS3)",
    "Physiology in Health, Disease and Ageing (LS4)",
    "Neuroscience and Disorders of the Nervous System (LS5)",
    "Immunity, Infection and Immunotherapy (LS6)",
    "Prevention, Diagnosis and Treatment of Human Diseases (LS7)",
    "Environmental Biology, Ecology and Evolution (LS8)",
    "Biotechnology and Biosystems Engineering (LS9)",
    "Individuals, Markets and Organisations (SH1)",
    "Institutions, Governance and Legal Systems (SH2)",
    "The Social World and Its Interactions (SH3)",
    "The Human Mind and Its Complexity (SH4)",
    "Texts and Concept (SH5)",
    "The Study of the Human Past (SH6)",
    "Human Mobility, Environment, and Space (SH7)",
    "Studies of Cultures and Arts (SH8)"
]

In [19]:
# display name of each subdomain
subdomain_names = df['Subdomain'].unique()
for subdomain in subdomain_names:
    display(subdomain)
    display(df[df['Subdomain'] == subdomain])

'Mathematics (PE1)'

,Domain,Subdomain,Forename,Lastname,Fullname,Type
0,PE,Mathematics (PE1),Sara,van de Geer,Sara van de Geer,Chair
1,PE,Mathematics (PE1),Elisabeth,Bouscaren,Elisabeth Bouscaren,Member
2,PE,Mathematics (PE1),Lucia,Caporaso,Lucia Caporaso,Member
3,PE,Mathematics (PE1),Alessandra,Celletti,Alessandra Celletti,Member
4,PE,Mathematics (PE1),Anthony,Davison,Anthony Davison,Member
5,PE,Mathematics (PE1),Pavel,Exner,Pavel Exner,Member
6,PE,Mathematics (PE1),Hélène,Frankowska,Hélène Frankowska,Member
7,PE,Mathematics (PE1),Friedrich,Götze,Friedrich Götze,Member
8,PE,Mathematics (PE1),Ansgar,Jüngel,Ansgar Jüngel,Member
9,PE,Mathematics (PE1),Radha,Kessar,Radha Kessar,Member


'Fundamental Constituents of Matter (PE2)'

,Domain,Subdomain,Forename,Lastname,Fullname,Type
15,PE,Fundamental Constituents of Matter (PE2),María,Borge,María José García Borge,Chair
16,PE,Fundamental Constituents of Matter (PE2),Cinzia,Da Via,Cinzia Da Via,Member
17,PE,Fundamental Constituents of Matter (PE2),Astrid,Eichhorn,Astrid Eichhorn,Member
18,PE,Fundamental Constituents of Matter (PE2),Wojciech,Gawlik,Wojciech Gawlik,Member
19,PE,Fundamental Constituents of Matter (PE2),Sven,Heinemeyer,Sven Heinemeyer,Member
20,PE,Fundamental Constituents of Matter (PE2),Ondřej,Klimo,Ondřej Klimo,Member
21,PE,Fundamental Constituents of Matter (PE2),José,Latorre,José Ignacio Latorre,Member
22,PE,Fundamental Constituents of Matter (PE2),Paolo,Mataloni,Paolo Mataloni,Member
23,PE,Fundamental Constituents of Matter (PE2),Giovanni,Modugno,Giovanni Modugno,Member
24,PE,Fundamental Constituents of Matter (PE2),Tatsuya,Nakada,Tatsuya Nakada,Member


'Condensed Matter Physics (PE3)'

...

,Domain,Subdomain,Forename,Lastname,Fullname,Type
384,SH,The Study of the Human Past (SH6),Francesca,Tinti,Francesca Tinti,Chair
385,SH,The Study of the Human Past (SH6),Diego,Angelucci,Diego E. Angelucci,Member
386,SH,The Study of the Human Past (SH6),Staša,Babić,Staša Babić,Member
387,SH,The Study of the Human Past (SH6),Pedro,Cardim,Pedro Cardim,Member
388,SH,The Study of the Human Past (SH6),Pam,Crabtree,Pam J. Crabtree,Member
389,SH,The Study of the Human Past (SH6),Eugénia,Cunha,Eugénia Cunha,Member
390,SH,The Study of the Human Past (SH6),Cesare,Cuttica,Cesare Cuttica,Member
391,SH,The Study of the Human Past (SH6),Véronique,Dasen,Véronique Dasen,Member
392,SH,The Study of the Human Past (SH6),Gabriel,Galvez-Behar,Gabriel Galvez-Behar,Member
393,SH,The Study of the Human Past (SH6),Niels,Gaul,Niels Gaul,Member


'Human Mobility, Environment, and Space (SH7)'

,Domain,Subdomain,Forename,Lastname,Fullname,Type
401,SH,"Human Mobility, Environment, and Space (SH7)",Ole,Mertz,Ole Mertz,Chair
402,SH,"Human Mobility, Environment, and Space (SH7)",Isabelle,Anguelovski,Isabelle Michele Sophie Anguelovski,Member
403,SH,"Human Mobility, Environment, and Space (SH7)",Antonio,Boggia,Antonio Boggia,Member
404,SH,"Human Mobility, Environment, and Space (SH7)",Maria,Brockhaus,Maria Brockhaus,Member
405,SH,"Human Mobility, Environment, and Space (SH7)",Daniel,Brown,Daniel Brown,Member
406,SH,"Human Mobility, Environment, and Space (SH7)",Basile,Chaix,Basile Chaix,Member
407,SH,"Human Mobility, Environment, and Space (SH7)",Audrey,de Nazelle,Audrey de Nazelle,Member
408,SH,"Human Mobility, Environment, and Space (SH7)",Martin,Dijst,Martin Dijst,Member
409,SH,"Human Mobility, Environment, and Space (SH7)",Aleksandra,Grzymała-Kazłowska,Aleksandra Grzymała-Kazłowska,Member
410,SH,"Human Mobility, Environment, and Space (SH7)",Stefanie,Möllborn,Stefanie Möllborn,Member


'Studies of Cultures and Arts (SH8)'

,Domain,Subdomain,Forename,Lastname,Fullname,Type
417,SH,Studies of Cultures and Arts (SH8),Matthew,Rampley,Matthew Rampley,Chair
418,SH,Studies of Cultures and Arts (SH8),Ana,da Silva Santos,Ana Cristina Alvarez Caiano da Silva Santos,Member
419,SH,Studies of Cultures and Arts (SH8),Martine,Beugnet,Martine Beugnet,Member
420,SH,Studies of Cultures and Arts (SH8),Erwan,Dianteill,Erwan Dianteill,Member
421,SH,Studies of Cultures and Arts (SH8),Noémie,Étienne,Noémie Étienne,Member
422,SH,Studies of Cultures and Arts (SH8),Ruurd,Halbertsma,Ruurd Binnert Halbertsma,Member
423,SH,Studies of Cultures and Arts (SH8),Katie,Holmes,Katie Holmes,Member
424,SH,Studies of Cultures and Arts (SH8),Jonathan,Impett,Jonathan Impett,Member
425,SH,Studies of Cultures and Arts (SH8),Shannon,Jackson,Shannon Jackson,Member
426,SH,Studies of Cultures and Arts (SH8),Alexander,Marr,Alexander Marr,Member
